In [ ]:
import os
import pickle
import sys

# Third-party
from astropy.io import ascii
import astropy.coordinates as coord
import astropy.table as table
import astropy.units as u
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as pl
pl.style.use('apw-notebook')
%matplotlib inline
from astropy.io.fits import getdata
from scipy.misc import logsumexp
from scipy.ndimage import gaussian_filter
from scipy import interpolate
from scipy.stats import scoreatpercentile
from astroML.utils import log_multivariate_gaussian
import h5py

In [ ]:
XCov_filename = "/Users/adrian/projects/globber/data/ngc5897/XCov_lg.h5"

In [ ]:
cluster_c = coord.SkyCoord(ra=229.352*u.degree,
                           dec=-21.01*u.degree)

In [ ]:
with h5py.File(XCov_filename, "r") as f:
    allX = f['search']['X'][:]
    pre_filter_ix = (allX[:,0] > 18.) & (allX[:,0] < 21.) & (allX[:,1] > 0.) & (allX[:,1] < 1.)
    
    allX = allX[pre_filter_ix]
    ra = f['search']['ra'][:][pre_filter_ix]
    dec = f['search']['dec'][:][pre_filter_ix]
    cluX = f['cluster']['X'][:]
    
all_c = coord.SkyCoord(ra=ra*u.degree, dec=dec*u.degree)

In [ ]:
pl.figure(figsize=(8,6))
pl.plot(ra, dec, ls='none', marker=',', alpha=0.05)
pl.gca().set_aspect('equal')
pl.xlim(pl.xlim()[::-1])

In [ ]:
cluster_ix = all_c.separation(cluster_c) < 10*u.arcmin

In [ ]:
def search_field(ra, dec):
    ix1 = (ra > 222) & (ra < 235) & (dec < -17) & (dec > -24)
    return ix1
search_ix = search_field(ra, dec)

In [ ]:
# def control_field(ra, dec):
#     ix1 = (ra > 228) & (ra < 232) & (dec < -24) & (dec > -26)
#     ix2 = (ra > 228) & (ra < 232) & (dec < -17) & (dec > -19)
#     return ix1 | ix2
# control_ix = control_field(ra, dec)

control_ix = search_ix & np.logical_not(cluster_ix)

In [ ]:
pl.figure(figsize=(8,6))

pl.plot(ra, dec, ls='none', marker=',', alpha=0.1)
pl.plot(ra[search_ix], dec[search_ix], ls='none', marker=',', alpha=0.1, color='g')
pl.plot(ra[control_ix], dec[control_ix], ls='none', marker=',', alpha=0.1, color='r')

pl.gca().set_aspect('equal')
pl.xlim(235,220)
pl.ylim(-26,-16)

In [ ]:
searchX = allX[search_ix]

---

In [ ]:
gi_step = 0.04
i_step = 0.08
gi_bins = np.arange(0.0,0.5+step,gi_step)
i_bins = np.arange(18,21+step,i_step)

cluster_H,g_edges,gi_edges = np.histogram2d(cluX[:,1], cluX[:,0], bins=(gi_bins, i_bins))
control_H,g_edges,gi_edges = np.histogram2d(allX[control_ix,1], allX[control_ix,0], bins=(gi_bins, i_bins))

i_mesh,gi_mesh = np.meshgrid((i_bins[1:]+i_bins[:-1])/2, (gi_bins[1:]+gi_bins[:-1])/2)

In [ ]:
print(cluster_H.min(), cluster_H.max())
print(control_H.min(), control_H.max())

In [ ]:
fig,axes = pl.subplots(1,3,figsize=(10,6),sharex=True,sharey=True)

ax = axes[0]
ax.pcolormesh(gi_mesh, i_mesh, cluster_H, cmap='Blues')
ax.set_xlim(gi_bins.min(), gi_bins.max())
ax.set_ylim(i_bins.max(), i_bins.min())
ax.set_xlabel('$g-i$')
ax.set_ylabel('$i$')

ax = axes[1]
ax.pcolormesh(gi_mesh, i_mesh, control_H, cmap='Blues')
ax.set_xlabel('$g-i$')

ax = axes[2]

div = (cluster_H / cluster_H.sum()) / (control_H / control_H.sum())
div[~np.isfinite(div)] = 0.
# div = gaussian_filter(div, sigma=[0.04/i_step,0.04/gi_step][::-1])
ax.pcolormesh(gi_mesh, i_mesh, div, cmap='Blues')
ax.set_xlabel('$g-i$')

# fig.colorbar()

In [ ]:
spl = interpolate.SmoothBivariateSpline(gi_mesh.ravel(), i_mesh.ravel(), 
                                        control_H.ravel()/control_H.sum(), kx=5, ky=5)
spl_control_H = spl.ev(gi_mesh.ravel(), i_mesh.ravel())
spl_control_H = spl_control_H.reshape(gi_mesh.shape)

In [ ]:
fig,axes = pl.subplots(1,3,figsize=(10,6),sharex=True,sharey=True)

ax = axes[0]
ax.pcolormesh(gi_mesh, i_mesh, cluster_H, cmap='Blues')
ax.set_xlim(gi_bins.min(), gi_bins.max())
ax.set_ylim(i_bins.max(), i_bins.min())
ax.set_xlabel('$g-i$')
ax.set_ylabel('$i$')

ax = axes[1]
ax.pcolormesh(gi_mesh, i_mesh, spl_control_H, cmap='Blues')
ax.set_xlabel('$g-i$')

ax = axes[2]

div = (cluster_H/cluster_H.sum()) / spl_control_H
# div[~np.isfinite(div)] = 0.
# # div = gaussian_filter(div, sigma=[0.04/i_step,0.04/gi_step][::-1])
ax.pcolormesh(gi_mesh, i_mesh, div, cmap='Blues', vmin=0, vmax=10)
ax.set_xlabel('$g-i$')

In [ ]:
matched_filter = div.copy()
matched_filter[(gi_mesh < 0.101) | (i_mesh < 18.85)] = 0.
matched_filter = gaussian_filter(matched_filter, sigma=[0.02/i_step,0.02/gi_step][::-1])

In [ ]:
fig,ax = pl.subplots(1,1,figsize=(3,6))

ax.pcolormesh(gi_mesh, i_mesh, matched_filter, cmap='Blues')
ax.set_xlim(gi_bins.min(), gi_bins.max())
ax.set_ylim(i_bins.max(), i_bins.min())
ax.set_xlabel('$g-i$')
ax.set_ylabel('$i$')

In [ ]:
print(search_ix.sum())

In [ ]:
weights = np.zeros(search_ix.sum())
for i in range(weights.size):
    derp,_,_ = np.histogram2d(searchX[i:i+1,1], searchX[i:i+1,0], 
                              bins=(gi_bins, i_bins))
    weights[i] = (derp * matched_filter).sum()
    if (i % 10000) == 0:
        print(i)

In [ ]:
search_H,g_edges,gi_edges = np.histogram2d(searchX[:,1], searchX[:,0], 
                                           bins=(gi_bins, i_bins), weights=weights)

In [ ]:
fig,ax = pl.subplots(1,1,figsize=(3,6))

ax.pcolormesh(gi_mesh, i_mesh, search_H, cmap='Blues')
ax.set_xlim(gi_bins.min(), gi_bins.max())
ax.set_ylim(i_bins.max(), i_bins.min())
ax.set_xlabel('$g-i$')
ax.set_ylabel('$i$')

---

In [ ]:
sky_binsize = (6*u.arcmin).to(u.degree).value
sky_smooth = (8*u.arcmin).to(u.degree).value / sky_binsize

search_ra = ra[search_ix]
search_dec = dec[search_ix]
ra_bins = np.arange(search_ra.min(), search_ra.max()+sky_binsize, sky_binsize)
dec_bins = np.arange(search_dec.min(), search_dec.max()+sky_binsize, sky_binsize)

In [ ]:
search_H_sky,ra_edges,dec_edges = np.histogram2d(search_ra, search_dec,
                                                 bins=(ra_bins, dec_bins), 
                                                 weights=weights)

unw_search_H_sky,ra_edges,dec_edges = np.histogram2d(search_ra, search_dec,
                                                     bins=(ra_bins, dec_bins))

search_H_sky = search_H_sky.T
unw_search_H_sky = unw_search_H_sky.T
# search_H_sky /= unw_search_H_sky

ra_mesh,dec_mesh = np.meshgrid((ra_edges[1:]+ra_edges[:-1])/2, (dec_edges[1:]+dec_edges[:-1])/2)

In [ ]:
H_operation = lambda x: x**2

In [ ]:
tmp = H_operation(search_H_sky.ravel())
bins = np.linspace(*scoreatpercentile(tmp, [1,99]), num=32)
pl.hist(tmp, bins=bins);

vmin,vmax = scoreatpercentile(tmp, [10,75])
pl.axvline(vmin, color='r')
pl.axvline(vmax, color='r')

In [ ]:
fig,axes = pl.subplots(1,2,figsize=(15,6),sharex=True,sharey=True)

ax = axes[0]
ax.pcolormesh(ra_mesh, dec_mesh, H_operation(search_H_sky), 
              cmap='Greys', vmin=vmin, vmax=vmax)

pa = mpl.patches.Circle((cluster_c.ra.degree, cluster_c.dec.degree), radius=(7*u.arcmin).to(u.degree).value,
                        edgecolor='r', facecolor='none')
ax.add_patch(pa)

ax.set_xlim(ra_mesh.max(), ra_mesh.min())
ax.set_ylim(dec_mesh.min(), dec_mesh.max())
ax.set_xlabel('RA [deg]')
ax.set_ylabel('Dec [deg]')
ax.set_aspect('equal')

ax = axes[1]
ax.pcolormesh(ra_mesh, dec_mesh, gaussian_filter(H_operation(search_H_sky), sky_smooth), 
              cmap='Greys', vmin=vmin, vmax=vmax)

pa = mpl.patches.Circle((cluster_c.ra.degree, cluster_c.dec.degree), radius=(7*u.arcmin).to(u.degree).value,
                        edgecolor='r', facecolor='none')
ax.add_patch(pa)

ax.set_xlim(ra_mesh.max(), ra_mesh.min())
ax.set_ylim(dec_mesh.min(), dec_mesh.max())
ax.set_xlabel('RA [deg]')
ax.set_aspect('equal')

In [ ]:
tmp = np.log10(search_H_sky.ravel())
fig,ax = pl.subplots(1,1,figsize=(6,6),sharex=True,sharey=True)

ax.contour(ra_mesh, dec_mesh, search_H_sky, levels=10**np.linspace(tmp.max()-2.,tmp.max(),12), colors='k') # cmap='magma_r',

# ax.set_xlim(ra_mesh.max(), ra_mesh.min())
# ax.set_ylim(dec_mesh.min(), dec_mesh.max())

ax.set_xlim(cluster_c.ra.degree + 1, cluster_c.ra.degree - 1)
ax.set_ylim(cluster_c.dec.degree - 1, cluster_c.dec.degree + 1)

ax.set_xlabel('RA [deg]')
ax.set_ylabel('Dec [deg]')
ax.set_aspect('equal')